In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [14]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [21]:
dependent=dataset[[ 'charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [22]:
#sklearn library used to split train and test model
from sklearn.model_selection import train_test_split
#Then call train_test_split function it is parameterised function
X_train,X_test,Y_train,Y_test=train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [23]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor(n_estimators=100,random_state=0,criterion="mae",max_features="sqrt")
regressor=regressor.fit(X_train,Y_train)

C:\Users\spavi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid={"criterion":['mse','mae'],'max_features':['auto','log2','sqrt'],'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)
#fitting the model for grid search
grid.fit(X_train,Y_train)

C:\Users\spavi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   35.4s finished
C:\Users\spavi\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['mse', 'mae'],

In [25]:
#print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print (re)
grid_predictions=grid.predict(X_test)
#print classification report
from sklearn.metrics import r2_score
r_score=r2_score(Y_test,grid_predictions)
print("The R_score value for best parameter{}:".format(grid.best_params_),r_score)

The R_score value for best parameter{'criterion': 'mse', 'max_features': 'log2', 'n_estimators': 100}: 0.8715675511988605


In [26]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.468509,0.174208,0.015621,0.022092,mse,auto,10,"{'criterion': 'mse', 'max_features': 'auto', '...",0.774415,0.795129,0.772631,0.780725,0.010211,10
1,0.728898,0.081579,0.019580,0.005599,mse,auto,100,"{'criterion': 'mse', 'max_features': 'auto', '...",0.804306,0.808197,0.795926,0.802810,0.005120,5
2,0.094397,0.022449,0.014295,0.004099,mse,log2,10,"{'criterion': 'mse', 'max_features': 'log2', '...",0.743474,0.800587,0.784544,0.776202,0.024051,11
3,0.512704,0.015241,0.029366,0.019433,mse,log2,100,"{'criterion': 'mse', 'max_features': 'log2', '...",0.817109,0.807742,0.799139,0.807997,0.007339,1
4,0.085165,0.019162,0.005207,0.007364,mse,sqrt,10,"{'criterion': 'mse', 'max_features': 'sqrt', '...",0.781883,0.773522,0.792395,0.782600,0.007721,8
5,0.475349,0.041530,0.045224,0.023296,mse,sqrt,100,"{'criterion': 'mse', 'max_features': 'sqrt', '...",0.807237,0.808353,0.795076,0.803555,0.006013,3
6,0.340335,0.031637,0.030004,0.021270,mae,auto,10,"{'criterion': 'mae', 'max_features': 'auto', '...",0.781562,0.771573,0.769270,0.774135,0.005336,12
7,4.210128,0.132181,0.080186,0.002941,mae,auto,100,"{'criterion': 'mae', 'max_features': 'auto', '...",0.804640,0.794056,0.789419,0.796038,0.006370,6
8,0.359369,0.021044,0.001995,0.002821,mae,log2,10,"{'criterion': 'mae', 'max_features': 'log2', '...",0.790799,0.786888,0.768016,0.781901,0.009947,9
9,2.739380,0.105694,0.035269,0.022806,mae,log2,100,"{'criterion': 'mae', 'max_features': 'log2', '...",0.806464,0.809130,0.794758,0.803451,0.006243,4


In [27]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age:45
BMI:67
Children:2
Sex Male 0 or 1:1
Smoker Yes 0 or 1:1


In [28]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,
smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[45528.574738]
